<a href="https://colab.research.google.com/github/LucasMatuszewski/Python-colab-notebooks/blob/main/CrewAI_Autonomus_Agents_Team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crew AI - Multi Agent
https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/



## Install dependencies

In [2]:
!pip install crewai
!pip install 'crewai[tools]'
!pip install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 629.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

## Choose Platform, Model and get API Key

In [3]:
import os
from google.colab import userdata
secretName = "MistralKey" # @param ["MistralKey", "OpenAIkey"] {type:"string"}
my_api_key = userdata.get(secretName)
os.environ['OPENAI_API_KEY'] = my_api_key

## Set Agents and Topic

In [ ]:
from crewai import Agent
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

# Topic for the crew run
topic = 'AI in healthcare' # @param {type: "string"}

# Creating a senior researcher agent with memory and verbose mode
agent1_role='Senior Researcher' # @param {type: "string"}
agent1_goal="Uncover groundbreaking technologies in" # @param {type: "string"}
agent1_backstory = "Driven by curiosity, you're at the forefront of innovation, eager to explore and share knowledge that could change the world." # @param {type: "string"}

researcher = Agent(
  role=agent1_role,
  goal=f'{agent1_goal} {topic}',
  verbose=True, # @param {type: "boolean"}
  memory=True, # @param {type: "boolean"}
  backstory=f'"""{agent1_backstory}"""',
  tools=[search_tool],
  allow_delegation=True # @param {type: "boolean"}
)

# Creating a writer agent with custom tools and delegation capability
agent2_role='Writer',# @param {type: "string"}
agent2_goal="agent2_goal}Narrate compelling tech stories about" # @param {type: "string"}
agent2_backstory="With a flair for simplifying complex topics, you craft engaging narratives that captivate and educate, bringing new discoveries to light in an accessible manner." # @param {type: "string"}

writer = Agent(
  role=agent2_role,
  goal=f'{agent2_goal} {topic}',
  verbose=True, # @param {type: "boolean"}
  memory=True, # @param {type: "boolean"}
  backstory=f'"""{agent2_backstory}"""',
  tools=[search_tool],
  allow_delegation=False # @param {type: "boolean"}
)

## Set Tasks for Agents

In [ ]:
from crewai import Task

# Research task
research_task = Task(
  description=f"""Identify the next big trend in {topic}.
  Focus on identifying pros and cons and the overall narrative.
  Your final report should clearly articulate the key points,
  its market opportunities, and potential risks.""",
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=f"""Compose an insightful article on {topic}.
  Focus on the latest trends and how it's impacting the industry.
  This article should be easy to understand, engaging, and positive.""",
  expected_output=f'A 4 paragraph article on {topic} advancements fromated as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

## Form the Crew

In [ ]:
from crewai import Crew, Process

# Forming the tech-focused crew with enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential  # Optional: Sequential task execution is default
)

## Kick It Off

In [ ]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff()
print(result)